In [4]:
import dotenv
import os
import re
from collections import Counter
from unidecode import unidecode
from tqdm import tqdm

import geopandas as gpd
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd

from ftlangdetect import detect
from geopy.geocoders import Nominatim, GoogleV3
from shapely.geometry import Point
from thefuzz import fuzz, process
from transformers import pipeline

from pocketknife.database import (
    connect_database, read_from_database)

from success_prediction.config import (
    PROJ_ROOT, RAW_DATA_DIR, EXTERNAL_DATA_DIR, PROCESSED_DATA_DIR)
from success_prediction.zefix_processing.clustering import PersonClustering

dotenv_path = os.path.join(PROJ_ROOT, '.env')
dotenv.load_dotenv(dotenv_path)

pd.set_option('future.no_silent_downcasting', True)

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

/Users/manuelbolz/Documents/git/for_work/company_success_prediction/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-07 22:55:24.888 | INFO     | success_prediction.config:<module>:11 - PROJ_ROOT path is: /Users/manuelbolz/Documents/git/for_work/company_success_prediction
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/manuelbolz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/manuelbolz/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/manuelbolz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# **PREPARE INSCRIBED PEOPLE/FIRMS DATA**

In [37]:
query_inscribed_people = """ 
    SELECT * FROM zefix.history_inscribed_people WHERE founders = TRUE AND shab_date < '2024-01-01';
"""

query_inscribed_firms = """ 
    SELECT * FROM zefix.history_inscribed_firms WHERE shab_date < '2024-01-01';
"""

with connect_database() as con:
    df_insc_people = read_from_database(connection=con, query=query_inscribed_people)
    df_insc_firms = read_from_database(connection=con, query=query_inscribed_firms)

In [377]:
def group_fids_with_combine_first(df, group_cols=['ehraid', 'fid']):
    """
    Combines instances of the same fid within a company and fills missing
    to create a complete representation for each founder
    """
    filled_rows = []
    for _, group in df.groupby(group_cols, sort=False):
        # Use first entry as base instance of fid
        combined = group.iloc[0]
        # Iteratively combine_first with the next row(s)
        for i in range(1, len(group)):
            combined = combined.combine_first(group.iloc[i])
        filled_rows.append(combined)
    return pd.DataFrame(filled_rows).reset_index(drop=True)

def build_founder_dict(df):
    founder_dict = {}
    for _, row in df.iterrows():
        ehraid = row['ehraid']
        fid = row['fid']
        
        # Handle BFS code lists for hometown (1–5)
        hometown_bfs_codes_latest = [
            int(code) for code in [
                row.get('hometown_1_bfs_gmde_code_latest'),
                row.get('hometown_2_bfs_gmde_code_latest'),
                row.get('hometown_3_bfs_gmde_code_latest'),
                row.get('hometown_4_bfs_gmde_code_latest'),
                row.get('hometown_5_bfs_gmde_code_latest'),
            ] if pd.notnull(code)
        ]
        
        # Handle BFS code lists for places of residence (1–2)
        place_of_residence_bfs_codes_latest = [
            int(code) for code in [
                row.get('place_of_residence_1_bfs_gmde_code_latest'),
                row.get('place_of_residence_2_bfs_gmde_code_latest'),
            ] if pd.notnull(code)
        ]
        
        # Handle nationality codes (1–3)
        nationality_iso_codes = [
            str(code) for code in [
                row.get('nationality_1_iso_3166_1_alpha_2'),
                row.get('nationality_2_iso_3166_1_alpha_2'),
                row.get('nationality_3_iso_3166_1_alpha_2'),
            ] if pd.notnull(code) and code != np.nan
        ]
        
        # Prepare entry
        founder_data = {
            'first_name': row.get('first_name') or '',
            'last_name': row.get('last_name') or '',
            'first_name_norm': row.get('first_name_norm') or '',
            'last_name_norm': row.get('last_name_norm') or '',
            'gender': row.get('gender') or '',
            'job_title': row.get('job_title') or '',
            'dr_title': row.get('founder_with_academic_title'),
            'signing_rights': row.get('signing_rights') or '',
            'shares': row.get('shares') or '',
            'hometown_bfs_codes_latest': hometown_bfs_codes_latest,
            'place_of_residence_bfs_codes_latest': place_of_residence_bfs_codes_latest,
            'nationality_iso_codes': nationality_iso_codes,
        }
        
        # Insert into nested dictionary
        if ehraid not in founder_dict:
            founder_dict[ehraid] = {}
        founder_dict[ehraid][fid] = founder_data
    
    return founder_dict

def count_founders(founder_dict):
    stats = {}
    for ehraid, founders in founder_dict.items():
        total = 0
        male = 0
        female = 0
        swiss = 0
        foreign = 0
        dr = 0
        for fdata in founders.values():
            total += 1
            gender = fdata.get('gender', '')
            if gender == 'm':
                male += 1
            elif gender == 'f':
                female += 1
            nationalities = fdata.get('nationality_iso_codes', [])
            if 'CH' in nationalities:
                swiss += 1
            else:
                foreign += 1
            dr += fdata.get('dr_title', 0)

        stats[ehraid] = {
            'n_founders': total,
            'n_female_founders': female,
            'n_male_founders': male,
            'n_swiss_founders': swiss,
            'n_foreign_founders': foreign,
            'n_dr_titles': dr
        }
    return stats

def get_founder_lists(founder_dict):
    stats = {}
    for ehraid, founders in founder_dict.items():
        names = [(fdata.get('first_name_norm', ''), fdata.get('last_name_norm', '')) for fdata in founders.values()]
        fids = [k for k in founders.keys()]
        nationalities = [fdata.get('nationality_iso_codes', []) for fdata in founders.values()]
        hometowns = [fdata.get('hometown_bfs_codes_latest', []) for fdata in founders.values()]
        residencies = [fdata.get('place_of_residence_bfs_codes_latest', []) for fdata in founders.values()]

        stats[ehraid] = {
            'founder_names': names,
            'founder_fids': fids,
            'founder_nationalities': nationalities,
            'founder_hometowns': hometowns,
            'founder_residencies': residencies
        }
    return stats

In [ ]:
# Pre-process and cluster the dataframe
bfs_code_cols = [col for col in df_insc_people.columns if 'bfs_gmde_code_' in col]
df_insc_people[bfs_code_cols] = df_insc_people[bfs_code_cols].astype(str).replace('0', np.nan)
df_insc_people[bfs_code_cols] = df_insc_people[bfs_code_cols].replace('None', np.nan).replace('', np.nan)

# Cluster the people
clustering = PersonClustering(df_insc_people)
clustered_df = clustering.cluster()

In [ ]:
# Group fids within companies
filled_df = group_fids_with_combine_first(clustered_df)
for col in bfs_code_cols:
    filled_df[col] = filled_df[col].astype(float)

Cluster people within company: 100%|██████████| 357357/357357 [09:51<00:00, 604.60it/s]


In [359]:
doctor_titles = [
    r"dr\.?",
    r"doctor",
    r"doktor",
    r"prof\.?",
    r"ph\.?\s*d\.?",         # PhD / Ph.D.
    r"dphil\.?",             # DPhil (Oxford)
    r"sc\.?\s*d\.?",         # ScD / Sc.D.
    r"dsc\.?", r"drsc\.?",   # DSc / DrSc
    r"dr\.?\s*-?\s*ing\.?",  # Dr-Ing.
    r"dott\.?",  # Italian variants
    r"dottore",
    r"hdr", # French short forms
]
pattern = re.compile(rf"\b(?:{'|'.join(doctor_titles)})\b", flags=re.IGNORECASE)
filled_df['founder_with_academic_title'] = filled_df['first_name'].fillna('').str.contains(pattern) | filled_df['last_name'].fillna('').str.contains(pattern)
filled_df['founder_with_academic_title'] = filled_df['founder_with_academic_title'].astype(int)

In [378]:
# Build dictionary with founders
founder_dict = build_founder_dict(filled_df)

# Count the total founders, female foundes, and male founders
count_stats = count_founders(founder_dict)

# Get the names, nationalities, hometowns, and place of residency of all founders as flat lists
founder_lists = get_founder_lists(founder_dict)

In [ ]:
# Prepare the number of inscribed firms at founding
grouped_insc_firms = df_insc_firms.groupby(['ehraid', 'shab_date']).agg({'shab_id': 'count'}).reset_index().rename(columns={'shab_id': 'n_inscribed_firms', 'shab_date': 'firm_inscription_date'})
grouped_insc_firms = grouped_insc_firms.sort_values('firm_inscription_date').drop_duplicates(subset=['ehraid'])

# **PREPARE FIRM-LEVEL DATA**

In [2]:
id2legalform = {
    1: 'Sole proprietorship',  # Einzelunternehmen  ->  EXCLUDE
    2: 'General Partnership',  # Kollektivgesellschaft  ->  INCLUDE
    3: 'Corporation',  # Aktiengesellschaft  ->  INCLUDE
    4: 'Limited Liability Company',  # Gesellschaft mit beschränkter Haftung  ->  INCLUDE
    5: 'Cooperative',  # Genossenschaft  ->  EXCLUDE
    6: 'Association',  # Verein  ->  EXCLUDE
    7: 'Foundation',  # Stiftung  ->  EXCLUDE
    8: 'Public sector institution',  # Institut des öffentlichen Rechts  ->  EXCLUDE
    9: 'Branch',  # Zweigniederlassung  ->  EXCLUDE
    10: 'Limited Partnership',  # Kommanditgesellschaft  ->  INCLUDE
    11: 'Foreign branch',  # Zweigniederlassung einer ausl. Gesellschaft  ->  EXCLUDE
    12: 'Corporation with unlimited partners',  # Kommanditaktiengesellschaft  ->  INCLUDE
    13: 'Special legal form',  # Besondere Rechtsform  ->  EXCLUDE
    14: 'Ownership in undivided shares',  # Gemeinderschaft  ->  EXCLUDE
    15: 'Limited Partnership for collective investment schemes with a fixed capital',  # Investmentgesellschaft mit festem Kapital  ->  INCLUDE
    16: 'Limited Partnership for collective investment schemes with a variable capital',  # Investmentgesellschaft mit variablem Kapital  ->  INCLUDE
    17: 'Limited Partnership for collective investment schemes',  # Kommanditgesellschaft für kollektive Kapitalanlagen  ->  INCLUDE
    18: 'Non commercial power of attorney',  # Nichtkaufmännische Prokure  ->  EXCLUDE
    19: '(unknown)',  # (unbekannt)  ->  EXCLUDE
}

growth_oriented_legal_forms = [2, 3, 4, 10, 12, 15, 16, 17]

In [ ]:
# This query gets the sample of growth oriented firms that were founded between 2016 and current for the prediction sample

query_founded_firms = """ 
    SELECT
        base.ehraid,
        base.uid,

        -- Dissolution information
        base.delete_date,
        dissolution.shab_date as dissolution_date,
        dissolution.reason_for_dissolution,
        dissolution.liquidation,
        dissolution.bankruptcy,
        dissolution.other_exit,

        -- Names
        base.name AS current_name,
        founding_name.firm_name AS founding_name,

        -- Legal forms
        base.legal_form_id AS current_legal_form,
        legal_form.legal_form_id AS founding_legal_form,

        -- Purpose
        base.purpose_raw AS current_purpose,
        founding_purpose.purpose_raw AS founding_purpose,
        founding_purpose.purpose_clean AS founding_purpose_clean,

        -- Founding NOGA code
        founding_purpose.section_1_label AS founding_section,
        founding_purpose.sec.class_1_label AS founding_class,

        -- Current address
        COALESCE(address.street, '') || ' ' || COALESCE(address.house_number, '') AS current_street,
        address.town AS current_town,
        address.swiss_zip_code AS current_zip_code,
        address.country AS current_country,

        -- Founding address
        founding_address.street AS founding_street,
        founding_address.town AS founding_town,
        founding_address.postal_code AS founding_zip_code,
        founding_address.town_bfs_gmde_code_latest AS founding_bfs_code,

        -- Founding SHAB entry
        shab.shab_id,
        shab.shab_date AS founding_date,
        shab.message AS founding_message

    FROM zefix_release_159.base base

    -- Founding SHAB messages
    INNER JOIN (
        SELECT s.ehraid, s.shab_id, s.shab_date, s.message
        FROM zefix_release_159.shab s
        INNER JOIN zefix_release_159.shab_mutation sm ON s.shab_id = sm.shab_id
        WHERE sm.description = 'status.neu'
    ) AS shab ON base.ehraid = shab.ehraid

    -- Current address
    LEFT JOIN zefix_release_159.address address ON base.ehraid = address.ehraid

    -- Founding address
    LEFT JOIN (
        SELECT DISTINCT hfa.ehraid, hfa.street, hfa.postal_code, hfa.town, hfa.town_bfs_gmde_code_latest
        FROM zefix.history_firm_addresses hfa
        WHERE founding = TRUE
    ) AS founding_address ON base.ehraid = founding_address.ehraid

    -- Founding name
    LEFT JOIN (
        SELECT DISTINCT hfn.ehraid, hfn.firm_name
        FROM zefix.history_firm_names hfn
        WHERE hfn.founding = TRUE
    ) AS founding_name ON base.ehraid = founding_name.ehraid

    -- Founding purpose
    LEFT JOIN (
        SELECT DISTINCT hp.ehraid, hp.purpose_raw, sec.purpose as purpose_clean, sec.section_1_label, sec.class_1_label
        FROM zefix.history_purpose hp
        LEFT JOIN zefix.history_sector sec
        ON hp.ehraid = sec.ehraid AND hp.shab_id = sec.shab_id
        WHERE hp.founding_purpose = TRUE
    ) AS founding_purpose ON base.ehraid = founding_purpose.ehraid

    -- Founding legal form
    LEFT JOIN (
        SELECT DISTINCT hlf.ehraid, hlf.legal_form_id
        FROM zefix.history_founding_legal_form hlf
    ) AS legal_form ON base.ehraid = legal_form.ehraid

    -- Dissolution information
    LEFT JOIN (
        -- Only keep the last dissolution message as the final dissolution
        SELECT hd.ehraid, hd.shab_date, hd.reason_for_dissolution, hd.liquidation, hd.bankruptcy, hd.other_exit
        FROM (
            SELECT *,
                ROW_NUMBER() OVER (PARTITION BY ehraid ORDER BY shab_date DESC) AS rn
            FROM zefix.history_dissolutions
        ) hd
        WHERE hd.rn = 1
    ) AS dissolution ON base.ehraid = dissolution.ehraid

    -- Filter out irrelevant records
    WHERE
        NOT base.is_branch
        AND shab.shab_date < '2024-01-01'
        AND base.legal_form_id IN (2, 3, 4, 10, 12, 15, 16, 17)
        AND LOWER(base.name) NOT LIKE '%zweigniederlassung%'
        AND LOWER(base.name) NOT LIKE '%succursale%';
"""

In [704]:
with connect_database() as con:
    df_startups = read_from_database(connection=con, query=query_founded_firms)

df_startups['founding_date'] = pd.to_datetime(df_startups['founding_date'])

In [706]:
# Observed duplicates stem from entries having multiple new inscriptions in Zefix. -> Remove them from the sample because history seems to contain errors
display(df_startups[df_startups.duplicated(subset=['ehraid', 'founding_town'], keep=False)].uid.unique())
df_startups = df_startups.drop_duplicates(subset=['ehraid'], keep=False)
display(df_startups.shape)

array([], dtype=object)

(226559, 25)

### GEO ENCODE ADDRESS INFORMATION

In [ ]:
# Use the current information if the founding address is missing
df_startups['founding_street'] = df_startups['founding_street'].fillna(df_startups['current_street'])
df_startups['founding_zip_code'] = df_startups['founding_zip_code'].fillna(df_startups['current_zip_code'])
df_startups['founding_town'] = df_startups['founding_town'].fillna(df_startups['current_town'])

In [116]:
assert df_startups[df_startups['founding_street'].isna()].empty
assert df_startups[df_startups['founding_zip_code'].isna()].empty
assert df_startups[df_startups['founding_town'].isna()].empty

In [ ]:
nominatim_geolocator = Nominatim(
    user_agent="local_geocoder",
    domain="localhost:8080",
    scheme="http"
)
google_geolocator = GoogleV3(api_key=os.getenv('GOOGLE_GEOCODE_API_KEY'))


def geocode_address(nominatim_geolocator, google_geolocator, row):
    try:
        location = nominatim_geolocator.geocode({
            'street': row['founding_street'],
            'city': row['founding_town'],
            'postalcode': int(row['founding_zip_code']),
            'country': 'Schweiz'
        }, timeout=2)
        if location:
            return pd.Series([location.address, location.latitude, location.longitude])
        else:
            location = google_geolocator.geocode({
                'street': row['founding_street'],
                'city': row['founding_town'],
                'postalcode': int(row['founding_zip_code']),
                'country': 'Schweiz'
            }, timeout=1)
            if location:
                return pd.Series([location.address, location.latitude, location.longitude])
            return pd.Series([None, None, None])
    except Exception as e:
        print(f"Error: {e}")
        return pd.Series([None, None, None])

In [ ]:
df_startups[['geocoded_address', 'latitude', 'longitude']] = df_startups.apply(lambda row: geocode_address(nominatim_geolocator, google_geolocator, row), axis=1)

In [285]:
df_startups['founding_bfs_code'] = df_startups['founding_bfs_code'].astype(int)
df_startups['founding_zip_code'] = df_startups['founding_zip_code'].astype(int)

### DETERMINE BFS MUNICIPALITY CODE BY COORDINATES WHERE MISSING

In [372]:
gdf = gpd.read_file(EXTERNAL_DATA_DIR / 'geo_data' / 'swissBOUNDARIES3D_1_5_LV95_LN02.gdb', layer="TLM_HOHEITSGEBIET")
gdf = gdf.to_crs("EPSG:4326")
gdf = gdf[['geometry', 'BFS_NUMMER', 'EINWOHNERZAHL']]

df_startups = gpd.GeoDataFrame(
    df_startups,
    geometry=gpd.points_from_xy(df_startups['longitude'], df_startups['latitude']),
    crs="EPSG:4326"
)

df_startups = gpd.sjoin(df_startups, gdf, how="left", predicate="within")

# Replace where code is 0 (unmatched) or where it does not match the coordinates
df_startups.loc[df_startups['founding_bfs_code'] == 0, 'founding_bfs_code'] = pd.NA
df_startups.loc[(df_startups['founding_bfs_code'].astype(float) != df_startups['BFS_NUMMER']) & (~df_startups['BFS_NUMMER'].isna()), 'founding_bfs_code'] = pd.NA
df_startups['founding_bfs_code'] = df_startups['founding_bfs_code'].fillna(df_startups['BFS_NUMMER'])

In [374]:
df_startups['founding_bfs_code'] = df_startups['founding_bfs_code'].astype(int)

In [375]:
df_startups[df_startups.founding_bfs_code.astype(float) != df_startups.BFS_NUMMER][['founding_town', 'combined_address', 'founding_bfs_code', 'BFS_NUMMER']]

,founding_town,combined_address,founding_bfs_code,BFS_NUMMER
223603,Chiasso,"Via Henry Dunant 1, 6830 Chiasso",5250,NaN
223613,Morcote,"Via Isella 11, 6922 Morcote",5203,NaN
223639,Brusino Arsizio,"Via Lungolago 83, 6827 Brusino Arsizio",5160,NaN
224506,San Bernardino,"Residenza Mons Avium , appartamento 25, 6565 S...",3822,NaN
224785,La Tène,"route de Bellevue 7, 2074 La Tène",6513,NaN
225829,Roggwil TG,"Im Pünst 1, 9325 Roggwil TG",4431,NaN
225873,Bassins,"Chemin de Raulan 24, 1269 Bassins",5703,NaN
226046,Warth,"Kartause Ittingen, 8532 Warth",4621,NaN


In [ ]:
df_startups.drop(columns=['geometry', 'index_right', 'Unnamed: 0', 'BFS_NUMMER'], inplace=True)

### ADD MUNICIPALITY TYPOLOGY

In [ ]:
df_typology = pd.read_excel(EXTERNAL_DATA_DIR / 'geo_data' / 'Raumgliederungen.xlsx')
df_typology.drop(columns=['Gemeindename', 'Bezirksname', 'Kanton'], inplace=True)
df_typology = df_typology.rename(columns={'BFS Gde-nummer': 'founding_bfs_code', 'Bezirks-nummer': 'district_id', 'Kantons-nummer': 'canton_id', 'Stadt/Land-Typologie': 'urban_rural', 'Gemeindetypologie (9 Typen)': 'typology_9c', 'Gemeindetypologie (25 Typen)': 'typology_25c'})

In [ ]:
df_startups = df_startups.merge(df_typology, on='founding_bfs_code', how='left')
df_startups.rename(columns={'EINWOHNERZAHL': 'population'}, inplace=True)

In [389]:
df_startups[df_startups.canton_id.isna()]

,ehraid,uid,delete_date,reason_for_dissolution,liquidation,bankruptcy,other_exit,current_name,founding_name,current_legal_form,...,combined_address,geocoded_address,latitude,longitude,population,canton_id,Bezirks-nummer,urban_rural,typology_9c,typology_25c
2469,1255845,CHE395917849,2020-06-19,['Nachdem kein begründeter Einspruch gegen die...,False,False,True,Same Same GmbH in Liquidation,Same Same GmbH,4,...,"Seeplatz 1, 8820 Wädenswil","1, Seeplatz, Wädenswil, Bezirk Horgen, Zürich,...",47.228758,8.676404,0.0,NaN,NaN,NaN,NaN,NaN
80542,1389236,CHE291873431,NaN,NaN,NaN,NaN,NaN,MS Glärnisch AG,MS Glärnisch AG,3,...,"Seeplatz 1, 8820 Wädenswil","1, Seeplatz, Wädenswil, Bezirk Horgen, Zürich,...",47.228758,8.676404,0.0,NaN,NaN,NaN,NaN,NaN
223147,1310452,CHE338654358,NaN,['Mit Entscheid vom 07.01.2025 hat der Einzelr...,False,True,False,Peter Jegen GmbH in Liquidation,Peter Jegen GmbH,4,...,"Sagastrasse 3, 7214 Grüsch","Sägastrasse 3, 9495 Triesen, Liechtenstein",47.088149,9.522204,5532.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
# df_startups.to_csv(RAW_DATA_DIR / 'company_sample' / 'geo_coded_company_sample.csv', index=False)

In [ ]:
# df_startups = pd.read_csv(RAW_DATA_DIR / 'company_sample' / 'geo_coded_company_sample.csv')

### ADD STARTING CAPITAL TO COMPANY DATA

In [ ]:
# Get historical exchange rates
import yfinance as yf

exchange_rate_dfs = []
for symbol in ['EUR', 'GBP', 'USD']:
    ticker = yf.Ticker(f'{symbol}CHF=X')
    df_ticker = ticker.history(start='2016-01-01', end='2024-01-01')
    df_ticker['symbol'] = symbol
    exchange_rate_dfs.append(df_ticker)

In [ ]:
df_exchange_rates = pd.concat(exchange_rate_dfs).reset_index()[['Date', 'symbol', 'Open']]

df_exchange_rates = df_exchange_rates.rename(columns={'Date': 'founding_date'})
df_exchange_rates['founding_date'] = pd.to_datetime(df_exchange_rates['founding_date']).dt.date
df_exchange_rates['founding_date'] = pd.to_datetime(df_exchange_rates['founding_date'])

df_temp = pd.DataFrame({'founding_date': pd.date_range(start='2016-01-01', end='2024-01-01').tolist() * 3})
df_temp['symbol'] = ['EUR'] * int(len(df_temp) / 3) + ['GBP'] * int(len(df_temp) / 3) + ['USD'] * int(len(df_temp) / 3)

df_exchange_rates = df_temp.merge(df_exchange_rates, on=['founding_date', 'symbol'], how='left')
df_exchange_rates['Open'] = df_exchange_rates['Open'].ffill()
df_exchange_rates['symbol'] = df_exchange_rates['symbol'].ffill()

df_exchange_rates.to_csv(EXTERNAL_DATA_DIR / 'exchange_rates' / 'exchange_rates.csv', index=False)

In [439]:
query_capital = """ 
    SELECT * FROM zefix.history_registered_capital WHERE shab_date < '2024-01-01';
"""

In [663]:
with connect_database() as con:
    df_capital = read_from_database(connection=con, query=query_capital)

In [664]:
df_capital = df_capital.rename(columns={'shab_date': 'founding_date', 'currency_new': 'symbol'})
df_capital['founding_date'] = pd.to_datetime(df_capital['founding_date'])

mapping = {
    'Euro': 'EUR',
    'Eur': 'EUR',
    'EURO': 'EUR',
    '€': 'EUR',
    'fr': 'CHF',
    'Fr.': 'CHF',
    'CHE': 'CHF',
    '£': 'GBP',
    'US': 'USD'
}
df_capital['symbol'] = df_capital['symbol'].replace(mapping)

# Drop duplicate entries where we have libaration information do avoid duplicates before aggregation
df_capital = df_capital[~df_capital.duplicated(subset=['ehraid', 'founding_date'], keep=False) | (df_capital.duplicated(subset=['ehraid', 'founding_date'], keep=False) & ~(df_capital['keyword'].str.contains('liberierung|liberato|libéré', regex=True)))]

# Drop entries where the currency is not a common currency
df_capital = df_capital[df_capital.symbol.isin(['CHF', 'EUR', 'USD', 'GBP'])]

# Add exchange rates and convert registered capital
df_capital = df_capital.merge(df_exchange_rates, on=['symbol', 'founding_date'], how='left')
df_capital['Open'] = df_capital['Open'].fillna(1.0)
df_capital['capital_chf'] = df_capital['capital_new'].astype(float) * df_capital['Open'].astype(float)

# Aggregate capital into one value for registered capital
df_capital_agg = df_capital.groupby(['ehraid', 'founding_date']).agg({'capital_chf': 'sum'}).reset_index()

In [ ]:
df_startups = df_startups.merge(df_capital_agg[['ehraid', 'founding_date', 'capital_chf']], on=['ehraid', 'founding_date'], how='left')

In [694]:
df_startups.head()

,ehraid,uid,delete_date,reason_for_dissolution,liquidation,bankruptcy,other_exit,current_name,founding_name,current_legal_form,...,latitude,longitude,population,canton_id,district_id,urban_rural,typology_9c,typology_25c,capital_chf,company_url
0,1251325,CHE153193257,NaN,['Mit Urteil des Gerichtspräsidenten des Zivil...,True,False,False,Arlez Carrosserie GmbH in Liquidation,Arlez Carrosserie GmbH,4,...,47.460137,7.861180,6296.0,13.0,1304.0,2.0,21.0,217.0,20000.0,http://www.arlez-carrosserie.ch/
1,1251326,CHE392024369,2020-11-11,[],False,False,True,Vista Coaching GmbH in Liquidation,Vista Coaching GmbH,4,...,47.504062,7.724207,4700.0,13.0,1303.0,1.0,11.0,113.0,20000.0,http://vista-coaching.ch/
2,1251327,CHE473646370,NaN,NaN,NaN,NaN,NaN,Wissler Consulting GmbH,Wissler Consulting GmbH,4,...,47.523075,7.845789,941.0,13.0,1304.0,3.0,23.0,236.0,20000.0,no website available
3,1251328,CHE205344235,NaN,NaN,NaN,NaN,NaN,Wolf Regio GmbH,Wolf Regio GmbH,4,...,47.518085,7.603328,12304.0,13.0,1301.0,1.0,11.0,112.0,20000.0,http://wolfregio.ch/
4,1251329,CHE190527339,NaN,"['Mit Entscheid vom 27.09.2022 , 9.15 Uhr , ha...",False,True,False,AHAS GmbH in Liquidation,AHAS GmbH,4,...,47.036300,8.177812,7771.0,3.0,312.0,2.0,21.0,216.0,20000.0,no website available


### ADD WEBSITE URLS TO COMPANY DATA

In [ ]:
websites = pd.read_csv(RAW_DATA_DIR / 'company_urls' / 'urls.csv')
websites['ehraid'] = websites['ehraid'].astype(int)

assert websites[websites.duplicated(subset='ehraid', keep=False)].empty

In [ ]:
df_startups = df_startups.merge(websites[['ehraid', 'company_url']], on='ehraid', how='left')
print(f'Percentage of companies with found website: {len(df_startups[df_startups.company_url != 'no website available']) / len(df_startups) * 100}')

In [ ]:
df_startups[df_startups.company_url != 'no website available'].to_csv(RAW_DATA_DIR / 'company_sample' / 'company_sample_website.csv', index=False)
df_startups[df_startups.company_url == 'no website available'].to_csv(RAW_DATA_DIR / 'company_sample' / 'company_sample_no_website.csv', index=False)

In [ ]:
total = df_startups.shape[0]
num_website = df_startups[(df_startups.company_url != 'no website available')].shape[0]
num_no_website = df_startups[(df_startups.company_url == 'no website available')].shape[0]

num_exits = df_startups[(df_startups.liquidation | df_startups.bankruptcy | df_startups.other_exit)].shape[0]
num_exits_website = df_startups[(df_startups.company_url != 'no website available') & (df_startups.liquidation | df_startups.bankruptcy | df_startups.other_exit)].shape[0]
num_exits_no_website = df_startups[(df_startups.company_url == 'no website available') & (df_startups.liquidation | df_startups.bankruptcy | df_startups.other_exit)].shape[0]

In [ ]:
num_survival = df_startups[~(df_startups.liquidation | df_startups.bankruptcy | df_startups.other_exit)].shape[0]
num_survival_website = df_startups[(df_startups.company_url != 'no website available') & ~(df_startups.liquidation | df_startups.bankruptcy | df_startups.other_exit)].shape[0]
num_survival_no_website = df_startups[(df_startups.company_url == 'no website available') & ~(df_startups.liquidation | df_startups.bankruptcy | df_startups.other_exit)].shape[0]
print(f'Percentage of firms having website: {num_website / total:.4f}')
print(f'Percentage of exited firms having website: {num_exits_website / num_exits:.4f}')
print(f'Percentage of survived firms having website: {num_survival_website / num_survival:.4f}')

In [ ]:
print(f'Ratio of survival to exit: {num_survival / num_exits:.2f}')
print(f'Ratio of survival with website to exit with website: {num_survival_website / num_exits_website:.2f}')

In [ ]:
df_startups[~(df_startups.liquidation | df_startups.bankruptcy | df_startups.other_exit)]
df_startups[(df_startups.company_url != 'no website available') & ~(df_startups.liquidation | df_startups.bankruptcy | df_startups.other_exit)]
df_startups[(df_startups.company_url != 'no website available') & (df_startups.liquidation | df_startups.bankruptcy | df_startups.other_exit)]

# **FEATURE ENCODING**

In [59]:
df_startups = pd.read_csv(RAW_DATA_DIR / 'company_sample' / 'full_company_sample.csv')
df_startups['founding_date'] = pd.to_datetime(df_startups['founding_date'])
df_startups['dissolution_date'] = pd.to_datetime(df_startups['dissolution_date'])

In [65]:
df_startups[(df_startups['founding_date'] <= '2022-04-01') & (df_startups['company_url'] != 'no website available') & (df_startups['company_url'].notnull())].to_csv(RAW_DATA_DIR / 'company_sample' / 'sample_2022-04-01_website.csv', index=False)

In [68]:
float(-1e-4)

-0.0001

In [70]:
df_startups.columns

Index(['ehraid', 'uid', 'delete_date', 'dissolution_date',
       'reason_for_dissolution', 'liquidation', 'bankruptcy', 'other_exit',
       'current_name', 'founding_name', 'current_legal_form',
       'founding_legal_form', 'current_purpose', 'founding_purpose',
       'current_street', 'current_town', 'current_zip_code', 'current_country',
       'founding_street', 'founding_town', 'founding_zip_code',
       'founding_bfs_code', 'shab_id', 'founding_date', 'founding_message',
       'combined_address', 'geocoded_address', 'latitude', 'longitude',
       'population', 'canton_id', 'district_id', 'urban_rural', 'typology_9c',
       'typology_25c', 'capital_chf', 'company_url'],
      dtype='object')

### ENCODE BASIC FIRM CHARACTERISTICS

In [ ]:
def get_quantiles(df: pd.DataFrame, column: str, q: list[float] = [.2, .4, .6, .8]) -> int:
    quantiles = df[column].quantile(q)
    def get_numeric_quintile(target: int, quantiles: list[float]):
        for i, tau in enumerate(quantiles, start=1):
            if target <= tau:
                return i
        return len(quantiles) + 1
    df[f'{column}_quantiles_{len(quantiles) + 1}'] = df[column].apply(lambda x: get_numeric_quintile(x, quantiles))
    return df

In [ ]:
# 1. Encode legal form as dummies (1-N dummies)
df_startups = pd.concat([df_startups, pd.get_dummies(df_startups['founding_legal_form'], prefix='legal_form_binary', drop_first=True, dtype=int)], axis=1)

# 2. Encode NOGA sectors
pass

# 3. Encode typology_25c as dummies
df_startups = pd.concat([df_startups, pd.get_dummies(df_startups['typology_25c'], prefix='typology_25c', drop_first=True, dtype=int)], axis=1)

# 4. Encode typology_9c as dummies
df_startups = pd.concat([df_startups, pd.get_dummies(df_startups['typology_9c'], prefix='typology_9c', drop_first=True, dtype=int)], axis=1)

# 5. Encode urban-rural as dummies
df_startups = pd.concat([df_startups, pd.get_dummies(df_startups['urban_rural'], prefix='urban_rural', drop_first=True, dtype=int)], axis=1)

# 6. Encode population quintiles
# df_startups = get_quantiles(df_startups, 'population')
# df_startups = pd.concat([df_startups, pd.get_dummies(df_startups['population_quantiles_5'], prefix='population_quintiles', drop_first=True, dtype=int)], axis=1)

### ENCODE ADDRESS FEATURES

In [84]:
def encode_spatial_features(df, lat_col='latitude', lon_col='longitude', founding_col='founding_date', dissolution_col='dissolution_date'):
    # Ensure date columns are datetime
    df = df.copy()
    df[founding_col] = pd.to_datetime(df[founding_col])
    df[dissolution_col] = pd.to_datetime(df[dissolution_col])
    
    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df[lon_col], df[lat_col]),
        crs='EPSG:4326'
    )
    gdf = gdf.to_crs(epsg=3857)
    sindex = gdf.sindex

    same_address_counts = []
    firms_within_1km = []
    firms_within_2_5km = []
    firms_within_10km = []

    for idx, row in gdf.iterrows():
        founding_date_i = row[founding_col]

        # -- Same address (within 10m)
        for radius, store in [
            (10, same_address_counts),
            (1000, firms_within_1km),
            (2500, firms_within_2_5km),
            (10_000, firms_within_10km)
        ]:
            possible_matches_idx = list(sindex.intersection(row.geometry.buffer(radius).bounds))
            nearby = gdf.iloc[possible_matches_idx]
            candidates = nearby[
                (nearby[founding_col] <= founding_date_i) &
                (
                    nearby[dissolution_col].isna() |
                    (nearby[dissolution_col] > founding_date_i)
                )
            ]
            n_firms = candidates[candidates.geometry.distance(row.geometry) <= radius].shape[0] - 1  # exclude self
            store.append(n_firms)

    gdf['n_firms_same_address'] = same_address_counts
    gdf['n_firms_within_1km'] = firms_within_1km
    gdf['n_firms_within_2.5km'] = firms_within_2_5km
    gdf['n_firms_within_10km'] = firms_within_10km

    return gdf.drop(columns='geometry')

In [85]:
# Add two spacial variables for
# 1. Number of firms (including the firm itself) at the same location (within 10 meters)
# 2. Number of firms (including the firm itself) within 1km distance
# Both are calculated only considering existing firms at the time of founding!

df_startups = encode_spatial_features(df_startups)

### ENCODE FIRM NAME FEATUES

In [87]:
porter_stemmer = nltk.stem.PorterStemmer()

token_classifier = pipeline(
  model="ZurichNLP/swissbert-ner",
  aggregation_strategy="simple",
  device='mps'
)

UMLAUT_REPLACEMENTS = {
    'ä': 'ae',
    'ö': 'oe',
    'ü': 'ue',
}

Device set to use mps


In [ ]:
def normalize_words(text: str) -> str:
    if not isinstance(text, str):
        return ''
    for char, replacement in UMLAUT_REPLACEMENTS.items():
        text = text.replace(char, replacement)
    return unidecode(text.lower())

def get_language(text: str) -> str:
    language = detect(text)
    return language.get('lang', 'de')

def stem_text(text: str, lang_code: str) -> str:
    code2lang = {
        'de': 'german',
        'en': 'english',
        'fr': 'french',
        'it': 'italian'
    }
    language = code2lang.get(lang_code, 'german')
    stop_words = set(stopwords.words(language))
    tokens = nltk.tokenize.word_tokenize(text, language=language)
    return [porter_stemmer.stem(w) for w in tokens if not w.lower() in stop_words]

def batch_ner_tag_bps(texts: list[str], token_classifier: pipeline):
    batch_outputs = token_classifier(texts, batch_size=32)
    results = []
    for output in batch_outputs:
        people = [entry['word'] for entry in output if entry['entity_group'] == 'PER']
        locations = [entry['word'] for entry in output if entry['entity_group'] == 'LOC']
        results.append((people, locations))
    return results

def process_df_with_ner(df_lang: pd.DataFrame, token_classifier: pipeline, batch_size: int):
    texts = df_lang['founding_purpose'].fillna("").tolist()
    results = []

    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i + batch_size]
        results.extend(batch_ner_tag_bps(batch, token_classifier))

    df_lang[['people', 'locations']] = pd.DataFrame(results, index=df_lang.index)
    return df_lang

def contains_male_or_female_name(names, gendered_first_names):
    for name in [n.split() for n in names]:
        gender = name in gendered_first_names
        if gender:
            return 1
    return 0

In [ ]:
raumgliederungen = pd.read_excel(EXTERNAL_DATA_DIR / 'geo_data' / 'Raumgliederungen.xlsx')
municipality_names = [re.sub(r'\(.*?\)', '', normalize_words(name)).strip() for name in raumgliederungen['Gemeindename']]

In [284]:
with connect_database() as con:
    gendered_names = read_from_database(con, "SELECT * FROM zefix.founders_gender_mapping WHERE gender != 'u' AND request_type = 'first_name' AND probability >= 0.95")

In [285]:
gendered_names['split_names'] = gendered_names['name'].str.split()

certain_gender = gendered_names[gendered_names['split_names'].apply(len) == 1]
certain_gender = certain_gender[certain_gender['name'].str.isalpha() & (certain_gender['name'].str.len() >= 4)].copy()

female_names = set(certain_gender[certain_gender['gender'] == 'f']['name'].apply(normalize_words))
male_names = set(certain_gender[certain_gender['gender'] == 'm']['name'].apply(normalize_words))

In [286]:
# Get external data to create firm name and BPS features (e.g. includes female name)
official_male_names = pd.read_csv(EXTERNAL_DATA_DIR / 'newborn_names' / 'maennliche_vornamen.csv', encoding='ISO-8859-15', usecols=['Vorname'])
official_female_names = pd.read_csv(EXTERNAL_DATA_DIR / 'newborn_names' / 'weibliche_vornamen.csv', encoding='ISO-8859-15', usecols=['Vorname'])

official_male_names = set([normalize_words(name) for name in official_male_names['Vorname']])
official_female_names = set([normalize_words(name) for name in official_female_names['Vorname']])

female_names = female_names.union(official_female_names)
male_names = male_names.union(official_male_names)

In [287]:
print(len(female_names))
print(len(male_names))

7198
10433


In [91]:
# 1. Firm name length
df_startups['firm_name_length'] = df_startups['founding_name'].str.len()

In [ ]:
df_startups['founding_name_norm'] = df_startups['founding_name'].apply(normalize_words)

In [102]:
# 2. Contains swiss reference
swiss_terms = [
    "switzerland",
    "swiss", 
    "schweiz",  # Covers "schweiz", "schweizer", "schweizerische", etc.
    "swi",  # e.g. swica
    "sui",
    "suisse",
    "helvet",  # Covers Helvetia, Helvetica, etc.
    "confed",  # Covers Confederation, "confédération", "confederazione"
    "sviz",  # Covers "Svizzera", "Svizzero", "Svizzere", "Svizzeri", "svizra", etc.
    "eidgen",  # Covers "eidgenossenschaft", "eidgenössisch",
]

df_startups['founding_name_norm'] = df_startups['founding_name'].apply(normalize_words)
df_startups['firm_name_swiss_ref'] = df_startups['founding_name_norm'].str.contains('|'.join(swiss_terms)).astype(int)

In [ ]:
# 3. Contains holding reference
holding_terms = [
    "holding",
    "beteiligung",
    "participation",
    "partecipazion",
    "anteil",
    "capital",
    "kapital",
    "invest",
    "share",
    "aktie",
    "action",
    "azion"
]

df_startups['founding_name_norm'] = df_startups['founding_name'].apply(normalize_words)
df_startups['firm_name_swiss_ref'] = df_startups['founding_name_norm'].str.contains('|'.join(swiss_terms)).astype(int)

In [ ]:
# 4. Contains geographic term (municipality name)
def has_geographic_term(firm_name, geographic_terms):
    names = [n for n in firm_name.split() if len(n) > 2]
    if any(name in geographic_terms for name in names):
        return 1
    return 0
    
df_startups['firm_name_geog_ref'] = df_startups['founding_name_norm'].apply(lambda x: has_geographic_term(x, municipality_names))

In [120]:
df_startups.head()

,ehraid,uid,delete_date,dissolution_date,reason_for_dissolution,liquidation,bankruptcy,other_exit,current_name,founding_name,...,population_quintiles_5,n_firms_same_address,n_firms_within_1km,n_firms_within_2.5km,n_firms_within_10km,firm_name_length,founding_name_norm,firm_name_swiss_ref,firm_name_geog_ref,firm_name_founder_match
0,1251327,CHE473646370,NaN,NaT,NaN,NaN,NaN,NaN,Wissler Consulting GmbH,Wissler Consulting GmbH,...,0,0,0,0,1,23.0,wissler consulting gmbh,0,0,0
1,1251329,CHE190527339,NaN,2022-10-03,"['Mit Entscheid vom 27.09.2022 , 9.15 Uhr , ha...",False,True,False,AHAS GmbH in Liquidation,AHAS GmbH,...,0,0,1,1,1,9.0,ahas gmbh,0,0,0
2,1251336,CHE350451441,NaN,NaT,NaN,NaN,NaN,NaN,Roof & Terrace AG,Roof & Terrace AG,...,0,0,2,3,3,17.0,roof & terrace ag,0,0,0
3,1251338,CHE497156719,2024-03-22,2024-03-22,['Aktiven und Passiven ( Fremdkapital ) gehen ...,False,False,True,SP Ventures GmbH,SP Ventures GmbH,...,0,0,0,3,3,16.0,sp ventures gmbh,0,0,0
4,1251341,CHE256940465,NaN,NaT,NaN,NaN,NaN,NaN,"The Food Bus, Yanev & Co","The Food Bus , Yanev & Co",...,0,0,0,0,0,25.0,"the food bus , yanev & co",0,0,0


In [ ]:
# 5. Contains founder names
def has_founder_name(firm_name, founders):
    if founders:
        founder_name_list = []
        for fn, ln in founders.get('founder_names', []):
            fn = [n for n in re.split(r'[- ]', fn) if len(n) > 2]  # Avoid really short names to decrease likelihood of false positive
            ln = [n for n in re.split(r'[- ]', ln) if len(n) > 2]
            founder_name_list.extend(fn + ln)
        return int(any(name in firm_name for name in founder_name_list))
    return 0

df_startups['firm_name_founder_match'] = df_startups.apply(lambda row: has_founder_name(row['founding_name_norm'], founder_lists.get(row['ehraid'])), axis=1)

In [ ]:
# 6. Contains gendered name
def contains_male_or_female_name(names: list[str] | str, gendered_first_names):
    if isinstance(names, list):
        names = [n for name in names for n in name.split() if len(n) > 2]
    elif isinstance(names, str):
        names = names.split()
    for name in names:
        gender = name in gendered_first_names
        if gender:
            return 1
    return 0

df_startups['firm_name_male_match'] = df_startups['founding_name_norm'].apply(lambda x: contains_male_or_female_name(x, male_names))
df_startups['firm_name_female_match'] = df_startups['founding_name_norm'].apply(lambda x: contains_male_or_female_name(x, female_names))

### ENCODE BPS FEATURES

In [292]:
df_startups.loc[df_startups.founding_purpose.isna(), 'founding_purpose'] = df_startups['current_purpose']  # Use current purpose if founding purpose is missing
df_startups['bps_language'] = df_startups['founding_purpose'].apply(get_language)

corrections = {
    'cs': 'de',
    'en': 'de',
    'sv': 'de',
    'es': 'fr',
    'pt': 'de',
    'pl': 'de',
    'nl': 'de',
    'ca': 'it',
    'et': 'de'
}

df_startups['bps_language'] = df_startups['bps_language'].replace(corrections)

In [ ]:
# Apply stop-word removal and stemming
df_startups['bps_normalized'] = df_startups.apply(lambda row: stem_text(row['founding_purpose'], lang_code=row['bps_language']), axis=1)

In [ ]:
# 1. Encode bps length
df_startups['bps_length'] = df_startups['bps_normalized'].apply(lambda x: len(' '.join(x)))  # bps length

# 2. Encode mean word length
df_startups['bps_mean_word_length'] = [(1 / len(word_list)) * np.array([len(w) for w in word_list]).sum() for word_list in df_startups['bps_normalized']]  # average word length per bps

In [295]:
# 3. Get quantiles of length metrics
df_startups = get_quantiles(df_startups, 'bps_length')

In [296]:
# 4. Calculate LIX
def calculate_lix(word_list: list[str]):
    pct_above_six = len([w for w in word_list if len(w) > 6])  / len(word_list) * 100
    return len(word_list) + pct_above_six

df_startups['bps_lix'] = df_startups['bps_normalized'].apply(calculate_lix)

In [297]:
# 5. Word-frequency Features
word_dictionary_de = [token for sublist in df_startups[df_startups['bps_language'] == 'de']['bps_normalized'] for token in sublist if token.isalpha() and len(token) > 4]
word_freqencies_de = Counter(word_dictionary_de)

word_dictionary_fr = [token for sublist in df_startups[df_startups['bps_language'] == 'fr']['bps_normalized'] for token in sublist if token.isalpha() and len(token) > 4]
word_freqencies_fr = Counter(word_dictionary_fr)

word_dictionary_it = [token for sublist in df_startups[df_startups['bps_language'] == 'it']['bps_normalized'] for token in sublist if token.isalpha() and len(token) > 4]
word_freqencies_it = Counter(word_dictionary_it)

def compute_specificity_features(bps_tokens: list[str], lang_code: str) -> tuple[int, int, float]:
    if not isinstance(bps_tokens, list):
        print(bps_tokens)
    if lang_code == 'fr':
        word_freqencies = word_freqencies_fr
    elif lang_code == 'it':
        word_freqencies = word_freqencies_it
    else:
        word_freqencies = word_freqencies_de
    token_freqs = [word_freqencies[token] for token in bps_tokens if token in word_freqencies]
    if not token_freqs:
        return (0, 0, 0.0)

    total_tokens = sum(word_freqencies.values())
    # Normalize it by total number of tokens to account for differences between languages
    min_freq_norm, max_freq_norm = min(token_freqs) / total_tokens, max(token_freqs) / total_tokens

    ratio = min_freq_norm / max_freq_norm if max_freq_norm > 0 else 0.0
    return (min_freq_norm, max_freq_norm, ratio)

df_startups[['bps_min_word_freq_norm', 'bps_max_word_freq_norm', 'bps_freq_ratio_norm']] = df_startups.apply(lambda row: compute_specificity_features(row['bps_normalized'], row['bps_language']), axis=1).apply(pd.Series)

In [319]:
# 5. Encode geographic and name features
dfs = []
for language in ['de', 'fr', 'it']:
    token_classifier.model.set_default_language(f"{language}_CH")
    df_lang = df_startups[df_startups['bps_language'] == language].copy()
    df_lang = process_df_with_ner(df_lang, token_classifier, batch_size=128)

    df_lang['has_location'] = df_lang['locations'].apply(lambda x: int(len(x) > 0))
    df_lang['people'] = df_lang['people'].apply(lambda names: [normalize_words(name) for name in names])
    df_lang['has_male_name'] = df_lang['people'].apply(lambda x: contains_male_or_female_name(x, male_names))
    df_lang['has_female_name'] = df_lang['people'].apply(lambda x: contains_male_or_female_name(x, female_names))

    dfs.append(df_lang)

df_startups = pd.concat(dfs)

100%|██████████| 1191/1191 [29:19<00:00,  1.48s/it]


TypeError: unhashable type: 'list'

### ENCODE FOUNDER FEATURES

In [405]:
# 1. Encode number of founders
df_startups['n_founders'] = df_startups['ehraid'].apply(lambda x: count_stats.get(x, {}).get('n_founders', 1))
df_startups.loc[df_startups['n_founders'] == 0, 'n_founders'] = 1  # Firm must have at least one founder

# 2. Encode number of inscribed firms
df_startups = df_startups.merge(grouped_insc_firms, on='ehraid', how='left')
df_startups['founding_date'] = pd.to_datetime(df_startups['founding_date'])
df_startups['firm_inscription_date'] = pd.to_datetime(df_startups['founding_date'])

df_startups.loc[df_startups['firm_inscription_date'] > df_startups['founding_date'], 'n_inscribed_firms'] = 0
df_startups.drop(columns=['firm_inscription_date'], inplace=True)
df_startups['n_inscribed_firms'].fillna(0, inplace=True)

# 3. Percentage of female founders
df_startups['n_female_founders'] = df_startups['ehraid'].apply(lambda x: count_stats.get(x, {}).get('n_female_founders', np.nan))
df_startups['pct_female_founders'] = df_startups['n_female_founders'] / df_startups['n_founders']

# 4. Number of distinct nationalities
def get_distinct_nationalities(founder_data):
    if not founder_data:
        return 0
    return len(set([nat for nationalities in founder_data.get('founder_nationalities', []) for nat in nationalities]))

df_startups['n_distinct_nationalities'] = df_startups['ehraid'].apply(lambda x: get_distinct_nationalities(founder_lists.get('ehraid')))

# 5. Number of Swiss founders
df_startups['n_swiss_founders'] = df_startups['ehraid'].apply(lambda x: count_stats.get(x, {}).get('n_swiss_founders', np.nan))

# 6. Number of foreign founders
df_startups['n_foreign_founders'] = df_startups['ehraid'].apply(lambda x: count_stats.get(x, {}).get('n_foreign_founders', np.nan))

# 7. Number of founders with Dr. PhD. Prof. in name
df_startups['n_dr_titles'] = df_startups['ehraid'].apply(lambda x: count_stats.get(x, {}).get('n_dr_titles', np.nan))

# 8. Founders with same municipality than firm
def get_distinct_nationalities(firm_bfs_code, founder_data):
    if not founder_data:
        return 0
    n_founders_with_same_bfs_code = 0
    for founder_residencies in founder_data.get('founder_residencies', []):
        n_founders_with_same_bfs_code += int(firm_bfs_code in founder_residencies)
    return n_founders_with_same_bfs_code

df_startups['n_founders_same_residence'] = df_startups.apply(lambda row: get_distinct_nationalities(row['founding_bfs_code'], founder_lists.get(row['ehraid'])), axis=1)

/var/folders/ly/6qys5jn501gbqmkkkgmvpf0h0000gn/T/ipykernel_69153/3770396621.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_startups['n_inscribed_firms'].fillna(0, inplace=True)


In [416]:
# 9. Encode prior founding experience
df_startups['founder_fids'] = df_startups['ehraid'].apply(lambda x: founder_lists.get(x, {}).get('founder_fids', []))

prior_foundings = {}
for ehraid, data in founder_lists.items():
    founders = data.get('founder_fids', [])
    for fid in founders:
        if fid not in prior_foundings:
            prior_foundings[fid] = [ehraid]
        else:
            prior_foundings[fid].append(ehraid)

prior_founding = []
prior_failed = []
prior_existing = []
for i, row in tqdm(df_startups.iterrows(), total=df_startups.shape[0]):
    foundings = 0
    failed = set()
    existing = set()
    for fid in row['founder_fids']:
        matches = df_startups[
            df_startups['ehraid'].isin(prior_foundings[fid])
            & (df_startups['ehraid'] != row['ehraid'])
            & (df_startups['founding_date'] <= row['founding_date'])]
        if matches.empty:
            continue
        foundings += 1
        failed_ehraids = matches[matches['dissolution_date'] < row['founding_date']]['ehraid'].tolist()
        for eid in failed_ehraids:
            failed.add(eid)
        existing_ehraids = [ehraid for ehraid in matches['ehraid'] if ehraid not in failed_ehraids]
        for eid in existing_ehraids:
            existing.add(eid)

    prior_founding.append(foundings)
    prior_failed.append(len(failed))
    prior_existing.append(len(existing))

100%|██████████| 226553/226553 [09:19<00:00, 405.15it/s]


In [417]:
df_startups['n_founders_with_experience'] = prior_founding
df_startups['pct_founders_with_experience'] = df_startups['n_founders_with_experience'] / df_startups['n_founders']
df_startups['prior_failed'] = prior_failed
df_startups['prior_existing'] = prior_existing

In [420]:
df_startups.columns

Index(['ehraid', 'uid', 'delete_date', 'dissolution_date',
       'reason_for_dissolution', 'liquidation', 'bankruptcy', 'other_exit',
       'current_name', 'founding_name', 'current_legal_form',
       'founding_legal_form', 'current_purpose', 'founding_purpose',
       'current_street', 'current_town', 'current_zip_code', 'current_country',
       'founding_street', 'founding_town', 'founding_zip_code',
       'founding_bfs_code', 'shab_id', 'founding_date', 'founding_message',
       'combined_address', 'geocoded_address', 'latitude', 'longitude',
       'population', 'canton_id', 'district_id', 'urban_rural', 'typology_9c',
       'typology_25c', 'capital_chf', 'company_url', 'legal_form_binary_3.0',
       'legal_form_binary_4.0', 'legal_form_binary_5.0',
       'legal_form_binary_6.0', 'legal_form_binary_8.0',
       'legal_form_binary_10.0', 'legal_form_binary_12.0',
       'legal_form_binary_13.0', 'legal_form_binary_15.0',
       'legal_form_binary_16.0', 'legal_form_binary_1

In [ ]:
df_startups

In [424]:
df_startups.founding_bfs_code.nunique()

2099

### ADD CONTROL VARIABLES

In [ ]:
# Add the number of days that we have a history to inform the model
# about missing history for variables depending on the history (e.g. n-firms at address, prior founding experiance)
df_startups['founding_date'] = pd.to_datetime(df_startups['founding_date'])
min_date = df_startups['founding_date'].agg('min')
df_startups['days_of_prior_observations'] = (df_startups['founding_date'] - min_date).dt.days

# **PROCESS ADDITIONAL OUTPUT FEATURES**

### INVOLUNTARY EXIT TARGET

### ACQUISITION TARGET

In [36]:
# Consolidation mergers are determined statistically via a name similarity index
# The cutoff is set to 0.7

query_merger = """ 
    SELECT ehraid_acquirer, ehraid_acquiree, merger_date, name_similarity FROM zefix.merger_relation WHERE merger_date > '2016-01-01' AND name_similarity < 0.7;
"""

with connect_database() as con:
    df_merger = read_from_database(connection=con, query=query_merger)

In [37]:
df_merger

,ehraid_acquirer,ehraid_acquiree,merger_date,name_similarity
0,20661,842122,2018-07-31,0.328784
1,20661,808336,2020-10-06,0.321066
2,186653,1376873,2020-11-27,0.432677
3,519390,1329420,2023-06-23,0.337552
4,672075,882324,2024-06-27,0.294701
...,...,...,...,...
15099,402443,367910,2016-05-24,0.366375
15100,893657,87137,2022-09-13,0.408320
15101,1452892,1250987,2023-04-19,0.442617
15102,133745,991716,2016-07-04,0.288470


In [22]:
df_merger

,ehraid_acquirer,ehraid_acquiree,merger_date
0,20661,842122,2018-07-31
1,20661,808336,2020-10-06
2,186653,1376873,2020-11-27
3,519390,1329420,2023-06-23
4,672075,882324,2024-06-27
...,...,...,...
15099,402443,367910,2016-05-24
15100,893657,87137,2022-09-13
15101,1452892,1250987,2023-04-19
15102,133745,991716,2016-07-04


### FUNDING TARGET

In [47]:
funding_df = pd.read_csv(PROCESSED_DATA_DIR / 'funding_data' / 'startup-ch_funding.csv')

In [49]:
funding_df[~funding_df.ehraid.isna()]

,date,firm_name,amount_chf,type,location,firm_name_norm,firm_name_clean,ehraid,uid,legal_seat
0,2016-01-01,Oviva AG,800'000,Equity,Altendorf,oviva ag,oviva ag,1153456.0,CHE442270754,Altendorf
1,2016-01-01,SteriLux SA,100'000,Loan,Prilly,sterilux sa,sterilux sa,1191889.0,CHE197903656,Renens (VD)
2,2016-01-01,wefox Holding AG,5'500'000,Equity,Zürich,wefox holding ag,wefox holding ag,1201794.0,CHE375651476,Zürich
3,2016-01-09,rqmicro AG,3'700'000,Equity,Schlieren,rqmicro ag,rqmicro ag,1139718.0,CHE412218826,Schlieren
4,2016-01-14,Goodwall SA,2'100'000,Equity,Genève,goodwall sa,goodwall sa,1059251.0,CHE136959165,Genève
...,...,...,...,...,...,...,...,...,...,...
4595,2022-03-01,pier wallet,1'000'000,Equity,Oberwil b.Zug,pier wallet,pier wallet,1522371,CHE421267195,NaN
4596,2022-02-01,pier wallet,100'000,Grant,Oberwil b.Zug,pier wallet,pier wallet,1522371,CHE421267195,NaN
4597,2018-12-19,readydata AG (immoledo),Undisclosed,Equity,Zürich,readydata ag,readydata ag,974688,CHE115498230,NaN
4598,2020-09-17,the GO corporation (EGO Movement),3'000'000,Equity,Zurich,the go corporation,the go corporation,1217368,CHE439270305,NaN


### INNOVATION SUBSIDY TARGET

In [43]:
inno_df = pd.read_csv(PROCESSED_DATA_DIR / 'funding_data' / 'innosuisse_grants.csv')

In [44]:
inno_df

,department,nabs_policy_domain,project_id,project_number,project_title,project_url,granted_total_costs,abstract,start_date,end_date,contact_person,scientific_management,implementation_partner,canton,city,zip_code,street,firm_name_original,ehraid,uid
0,INNOSUISSE,Industrial production and technology,36489,17743.1 PFIW-IW,Research Proposal on Experimental and Numerica...,https://www.aramis.admin.ch/Beteiligte/?Projec...,85500.00,NaN,2016-01-01,2017-01-01,"[{'first_name': 'Reza', 'last_name': 'Abhari',...",[],"{'first_name': 'René', 'last_name': 'Hunziker'...",AG,Baden,5400.0,Bruggerstrasse 71a,ABB Turbo Systems AG,329381.0,CHE101538426
1,INNOSUISSE,Industrial production and technology,37239,17856.2 PFES-ES,Green spaces toolbox for high quality developm...,https://www.aramis.admin.ch/Beteiligte/?Projec...,336000.00,NaN,2016-02-01,2018-05-01,"[{'first_name': 'Joachim', 'last_name': 'Schöf...",[],"{'first_name': 'Heinz', 'last_name': 'Beiner',...",ZH,Zürich,8001.0,Obere Zäune 12,Planpartner AG,137360.0,CHE107822340
2,INNOSUISSE,Industrial production and technology,37290,18589.1 PFIW-IW,Design and development of an industrial proces...,https://www.aramis.admin.ch/Beteiligte/?Projec...,458403.00,NaN,2016-02-01,2018-11-01,"[{'first_name': 'Frédéric', 'last_name': 'Pich...",[],"{'first_name': 'Pascal', 'last_name': 'Queloz'...",NE,La Chaux-de-Fonds,2302.0,Louis-Joseph Chevrolet 46,OPAL création SA,180947.0,CHE101886525
3,INNOSUISSE,Industrial production and technology,37295,18022.1 PFLS-LS,Structure-based drug discovery with thermostab...,https://www.aramis.admin.ch/Beteiligte/?Projec...,886937.00,G protein-coupled receptors (GPCRs) are the si...,2016-03-01,2019-03-01,"[{'first_name': 'Andreas', 'last_name': 'Plück...",[],"{'first_name': 'Carlo', 'last_name': 'Bertozzi...",ZH,Schlieren,8952.0,Grabenstrasse 11a,Heptares Therapeutics Zurich AG,1146674.0,CHE488613942
4,INNOSUISSE,Industrial production and technology,37301,18536.1 PFLS-LS,Integrated control of Colletotrichum coccodes ...,https://www.aramis.admin.ch/Beteiligte/?Projec...,444840.00,NaN,2016-03-01,2019-09-01,"[{'first_name': 'Peter', 'last_name': 'Spring'...",[],"{'first_name': 'Ernst', 'last_name': 'Wullschl...",BE,Bern,3007.0,Belpstrasse 26,CH Branchenorganisation der,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7600,INNOSUISSE,Industrial production and technology,56829,123.400 IP-LS,"A Non-Toxic, Long Lasting Anti-Microbial Polym...",https://www.aramis.admin.ch/Beteiligte/?Projec...,736193.95,"We target a non-toxic (no harmful elements, e....",2025-03-01,2027-08-01,"[{'first_name': None, 'last_name': None, 'cant...","[{'first_name': None, 'last_name': None, 'cant...","{'first_name': None, 'last_name': None, 'canto...",NaN,NaN,NaN,NaN,Müri Prototech AG,908535.0,CHE114294676
7601,INNOSUISSE,Industrial production and technology,56829,123.400 IP-LS,"A Non-Toxic, Long Lasting Anti-Microbial Polym...",https://www.aramis.admin.ch/Beteiligte/?Projec...,736193.95,"We target a non-toxic (no harmful elements, e....",2025-03-01,2027-08-01,"[{'first_name': None, 'last_name': None, 'cant...","[{'first_name': None, 'last_name': None, 'cant...","{'first_name': None, 'last_name': None, 'canto...",NaN,NaN,NaN,NaN,TopCoat GmbH,634713.0,CHE109287995
7602,INNOSUISSE,Industrial production and technology,56829,123.400 IP-LS,"A Non-Toxic, Long Lasting Anti-Microbial Polym...",https://www.aramis.admin.ch/Beteiligte/?Projec...,736193.95,"We target a non-toxic (no harmful elements, e....",2025-03-01,2027-08-01,"[{'first_name': None, 'last_name': None, 'cant...","[{'first_name': None, 'last_name': None, 'cant...","{'first_name': None, 'last_name': None, 'canto...",NaN,NaN,NaN,NaN,EMS-CHEMIE AG,52213.0,CHE105904234
7603,INNOSUISSE,Industrial production and technology,56832,120.818 IP-LS,Identification of novel prognostic and monitor...,https://www.aramis.admin.ch/Beteiligte/?Projec...,856312.95,Current cancer monitoring methods are often in...,2024-11-15,